In [ ]:
import pandas as pd
import numpy as np
import re
from keras.models import load_model
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
from transformers import BertTokenizer


# Load data
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')


## build the model

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report

# Check if GPU is available and set memory growth to avoid memory allocation issues
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

# Load the datasets
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_length = 128  # Reduced max token length

# Encode the labels
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['new_label'])
test_data['label'] = label_encoder.transform(test_data['new_label'])

# Function to encode texts
def encode_texts(texts):
    return tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

# Encode the texts
train_encodings = encode_texts(train_data['text'])
test_encodings = encode_texts(test_data['text'])

# Extract labels
train_labels = train_data['label'].values
test_labels = test_data['label'].values

# Load the DistilBERT model
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# Define the DNN for classification
input_ids = tf.keras.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
cls_output = bert_outputs.last_hidden_state[:, 0, :]  # Take the CLS token output

# Add DNN layers
x = tf.keras.layers.Dense(256, activation='relu')(cls_output)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)  # Use the number of unique classes

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Convert the data into TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(64).prefetch(tf.data.AUTOTUNE)

# Train the model with XLA (Accelerated Linear Algebra) optimization
tf.config.optimizer.set_jit(True)  # Enable XLA

# Train the model
history = model.fit(train_dataset, validation_data=test_dataset, epochs=1)


In [ ]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Get predictions
y_pred = []
y_true = []

for batch in test_dataset:
    inputs, labels = batch
    y_true.extend(labels.numpy())
    y_pred_batch = model.predict(inputs)
    y_pred.extend(np.argmax(y_pred_batch, axis=1))

# Print classification report
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))